<a href="https://colab.research.google.com/github/safednn-nasa/Prophecy/blob/master/MNIST_Prophecy_w_Attribution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import math
import io
import os
from collections import namedtuple
import sys
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn import tree
from tqdm import tqdm
import operator
import pandas as pd

from tensorflow.examples.tutorials.mnist import input_data


In [0]:
#!python 
!pip3 install -U pybind11

Requirement already up-to-date: pybind11 in /usr/local/lib/python3.6/dist-packages (2.4.3)


In [0]:
BATCH_SIZE = 50
TEST_BATCH_SIZE = 1000  
IMAGE_SIZE = 28
NUM_CLASSES = 10

LAYER = 4
LABEL = 0


## **LOAD INPUT DATA FOR MNIST**

In [0]:
mnist = input_data.read_data_sets('MNIST_data')


def read_inputs_from_file(inputFile, height, width):
    global inputMatrix, labelMatrix, mnist_inp_images, mnist_inp_labels
    with open(inputFile) as f:
        lines = f.readlines()
        print(len(lines), "examples")
        inputMatrix = np.empty(len(lines),dtype=list)
        labelMatrix = np.zeros(len(lines),dtype=int)
        mnist_inp_images = np.zeros([8,784],dtype=float)
        mnist_inp_labels = np.zeros(8,dtype=int)
        for l in range(len(lines)):
            if (l == 8):
              break
            k = [float(stringIn) for stringIn in lines[l].split(',')[1:]] #This is to remove the useless 1 at the start of each string. Not sure why that's there.
            inputMatrix[l] = np.zeros((height, width, 1),dtype=float) #we're asuming that everything is 2D for now. The 1 is just to keep numpy happy.
            labelMatrix[l] = lines[l].split(',')[0]
            mnist_inp_labels[l] = labelMatrix[l]
            count = 0
            for i in range(height):
                for j in range(width):
                        print(count)
                        inputMatrix[l][i][j] = k[count]
                        print(k[count])
                        pixel_num = (i *28) + j
                        mnist_inp_images[l][pixel_num] = inputMatrix[l][i][j]
                        count += 1
 
#!wget https://raw.githubusercontent.com/safednn-nasa/prophecy_DNN/sym_convnn/master/MNIST_ITR_REL/mnist_inputs1.txt -O ./mnist_train_labels.txt

#read_inputs_from_file('./mnist_train_labels.txt', 28, 28)

   

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


##**ARCHITECTURE OF THE MNIST MODEL (10 LAYER) IN TENSORFLOW** 

In [0]:

def weight_variable(shape, name):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial, name=name)

def bias_variable(shape, name):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial, name=name)

def fc2d(x, W):
  return tf.nn.fc2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

def create_model():
    x = tf.identity(tf.placeholder(tf.float32, shape=[None, 784]), name="input")
    y_ = tf.placeholder(tf.float32, shape=[None, 10])
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    W_fc1 = weight_variable([784, 10],name='w_fc1')
    b_fc1 = bias_variable([10],name='b_fc1')
    x_image = tf.reshape(x, [-1, 784])
    h_fc1_relu_inp = tf.identity(tf.matmul(x_image, W_fc1) + b_fc1, name='h_fc1_relu_inp')
    h_fc1 = tf.nn.relu(tf.matmul(x_image, W_fc1) + b_fc1)
#    h_fc1_iden = tf.identity(h_fc1,name='h_fc1')
    
    
    W_fc2 = weight_variable([10, 10],name='w_fc2')
    b_fc2 = bias_variable([10],name='b_fc2')
    h_fc2_relu_inp = tf.identity(tf.matmul(h_fc1, W_fc2) + b_fc2, name='h_fc2_relu_inp')
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)
 #   h_fc2_iden = tf.identity(h_fc2,name='h_fc2')
    
    
    W_fc3 = weight_variable([10, 10],name='w_fc3')
    b_fc3 = bias_variable([10],name='b_fc3')
    h_fc3_relu_inp = tf.identity(tf.matmul(h_fc2, W_fc3) + b_fc3, name='h_fc3_relu_inp')
    h_fc3 = tf.nn.relu(tf.matmul(h_fc2, W_fc3) + b_fc3)
  #  h_fc3_iden = tf.identity(h_fc3,name='h_fc3')

    W_fc4 = weight_variable([10, 10],name='w_fc4')
    b_fc4 = bias_variable([10],name='b_fc4')
    h_fc4_relu_inp = tf.identity(tf.matmul(h_fc3, W_fc4) + b_fc4, name='h_fc4_relu_inp')
    h_fc4 = tf.nn.relu(tf.matmul(h_fc3, W_fc4) + b_fc4)
   # h_fc4_iden = tf.identity(h_fc4,name='h_fc4')
    
    
    W_fc5 = weight_variable([10, 10],name='w_fc5')
    b_fc5 = bias_variable([10],name='b_fc5')
    h_fc5_relu_inp = tf.identity(tf.matmul(h_fc4, W_fc5) + b_fc5, name='h_fc5_relu_inp')
    h_fc5 = tf.nn.relu(tf.matmul(h_fc4, W_fc5) + b_fc5)
    #h_fc5_iden = tf.identity(h_fc5,name='h_fc5')

   
    W_fc6 = weight_variable([10, 10],name='w_fc6')
    b_fc6 = bias_variable([10],name='b_fc6')
    h_fc6_relu_inp = tf.identity(tf.matmul(h_fc5, W_fc6) + b_fc6, name='h_fc6_relu_inp')
    h_fc6 = tf.nn.relu(tf.matmul(h_fc5, W_fc6) + b_fc6)
   # h_fc6_iden = tf.identity(h_fc6,name='h_fc6')
    
    
    W_fc7 = weight_variable([10, 10],name='w_fc7')
    b_fc7 = bias_variable([10],name='b_fc7')
    h_fc7_relu_inp = tf.identity(tf.matmul(h_fc6, W_fc7) + b_fc7, name='h_fc7_relu_inp')
    h_fc7 = tf.nn.relu(tf.matmul(h_fc6, W_fc7) + b_fc7)
   # h_fc7_iden = tf.identity(h_fc7,name='h_fc7')

    W_fc8 = weight_variable([10, 10],name='w_fc8')
    b_fc8 = bias_variable([10],name='b_fc8')
    h_fc8_relu_inp = tf.identity(tf.matmul(h_fc7, W_fc8) + b_fc8, name='h_fc8_relu_inp')
    h_fc8 = tf.nn.relu(tf.matmul(h_fc7, W_fc8) + b_fc8)
   # h_fc8_iden = tf.identity(h_fc8,name='h_fc8')
    
    
    W_fc9 = weight_variable([10, 10],name='w_fc9')
    b_fc9 = bias_variable([10],name='b_fc9')
    h_fc9_relu_inp = tf.identity(tf.matmul(h_fc8, W_fc9) + b_fc9, name='h_fc9_relu_inp')
    h_fc9 = tf.nn.relu(tf.matmul(h_fc8, W_fc9) + b_fc9)
   # h_fc9_iden = tf.identity(h_fc9,name='h_fc9')

    W_fc10 = weight_variable([10, 10],name='w_fc10')
    b_fc10 = bias_variable([10],name='b_fc10')
    h_fc10_relu_inp = tf.identity(tf.matmul(h_fc9, W_fc10) + b_fc10, name='h_fc10_relu_inp')
    h_fc10 = tf.nn.relu(tf.matmul(h_fc9, W_fc10) + b_fc10)
    #h_fc10_iden = tf.identity(h_fc10,name='h_fc10')

    W_fc11 = weight_variable([10, 10],name='w_fc11')
    b_fc11 = bias_variable([10],name='b_fc11')
    
    y_fc = tf.matmul(h_fc10, W_fc11) + b_fc11
    h_y_fc_iden = tf.identity(y_fc,name='y_fc')
    
    
    prediction = tf.identity(tf.nn.softmax(y_fc), name="import/prediction")
    
    
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_fc))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(y_fc, 1), tf.argmax(y_, 1))
    
    
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    

    return cross_entropy, accuracy, x, keep_prob, y_fc, y_, W_fc1, W_fc2, W_fc3, W_fc4, W_fc5, W_fc6, W_fc7, W_fc8, W_fc9, W_fc10, W_fc11, b_fc1, b_fc2, b_fc3, b_fc4, b_fc5, b_fc6, b_fc7, b_fc8, b_fc9, b_fc10, b_fc11, h_fc1, h_fc1_relu_inp, h_fc2, h_fc2_relu_inp, h_fc3, h_fc3_relu_inp, h_fc4, h_fc4_relu_inp, h_fc5, h_fc5_relu_inp, h_fc6, h_fc6_relu_inp, h_fc7, h_fc7_relu_inp, h_fc8, h_fc8_relu_inp, h_fc9, h_fc9_relu_inp, h_fc10, h_fc10_relu_inp, y_fc


### **Train a new model**

In [0]:
#tf.reset_default_graph()
#sess = tf.InteractiveSession()
#cross_entropy, accuracy, x, keep_prob, y_conv, y_ = create_model()
#train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
#saver = tf.train.Saver(write_version=tf.train.SaverDef.V2)
#sess.run(tf.global_variables_initializer())
#for i in range(0, 1200):
#  batch = mnist.train.next_batch(BATCH_SIZE)
#  train_step.run(feed_dict={x: batch[0], y_: np.eye(10)[batch[1]], keep_prob: 0.5})
#  if i%100 == 0:
#    test_accuracy = accuracy.eval(feed_dict={
#        x:mnist.test.images, y_: np.eye(10)[mnist.test.labels], keep_prob: 1.0})
#    print("step %d, test accuracy %g"%(i, test_accuracy))    
#ckpt_path_name = saver.save(sess, './checkpoints/mnist_invariant.ckpt', global_step=i)
#print "Checkpoint saved at: %s" % ckpt_path_name

In [0]:
#from google.colab import files
#files.download(ckpt_path_name + '.data-00000-of-00001')
#files.download(ckpt_path_name + '.index')
#files.download(ckpt_path_name + '.meta')

### Restore a pretrained model from check point

In [0]:
#!mkdir -p ./checkpoints
#!wget https://github.com/safednn-nasa/prophecy_DNN/sym_convnn/raw/master/MNIST_ITR_REL/MNIST_conv_checkpoint/mnist_invariants.ckpt.index -O ./checkpoints/mnist_invariants.ckpt.index
#!wget https://github.com/safednn-nasa/prophecy_DNN/sym_convnn/raw/master/MNIST_ITR_REL/MNIST_conv_checkpoint/mnist_invariants.ckpt.meta -O ./checkpoints/mnist_invariants.ckpt.meta
#!wget https://github.com/safednn-nasa/prophecy_DNN/sym_convnn/raw/master/MNIST_ITR_REL/MNIST_conv_checkpoint/mnist_invariants.ckpt.data-00000-of-00001 -O ./checkpoints/mnist_invariants.ckpt.data-00000-of-00001

##**Restore pre-trained model from a .nn file with weights and biases**

In [0]:
def read_weights_from_file(inputFile):
    global weightMatrix, biasMatrix
    
    with open(inputFile) as f:
        lines = f.readlines()
        for indx in range(0,len(lines)):
            #print(indx, lines[indx])
            numberOfLayers = int(lines[0])
            numberOfLayers = 11
            weightMatrix = np.empty(numberOfLayers, dtype=list)
            biasMatrix = np.empty(numberOfLayers, dtype=list)
            currentLine = 2
            for i in range(numberOfLayers):
              dimensions = lines[currentLine].split(',')
              dimensions = [int(stringDimension) for stringDimension in dimensions]
              #print dimensions
              currentLine += 1
              weights = [float(stringWeight) for stringWeight in lines[currentLine].split(',')]
              #print len(weights)
              count = 0
              weightMatrix[i] = np.zeros((dimensions[0], dimensions[1]), dtype=float)
              for j in range(dimensions[1]):
                 for k in range(dimensions[0]):
                      weightMatrix[i][k][j] = weights[count]
                      count += 1
              currentLine += 1
              biases = [float(stringBias) for stringBias in lines[currentLine].split(',')]
              biasMatrix[i] = np.zeros(len(biases))
              biasMatrix[i] = biases
              currentLine += 2

!wget https://raw.githubusercontent.com/safednn-nasa/prophecy_DNN/master/MNIST_ITR_REL/mnist_10_layer.txt -O ./mnist_10_layer.txt


read_weights_from_file('./mnist_10_layer.txt')

tf.reset_default_graph()
sess = tf.InteractiveSession()
cross_entropy, accuracy, x, keep_prob, y_fc, y_, W_fc1, W_fc2, W_fc3, W_fc4, W_fc5, W_fc6, W_fc7, W_fc8, W_fc9, W_fc10, W_fc11,b_fc1, b_fc2, b_fc3, b_fc4, b_fc5, b_fc6, b_fc7, b_fc8, b_fc9, b_fc10, b_fc11, h_fc1, h_fc1_relu_inp, h_fc2, h_fc2_relu_inp, h_fc3, h_fc3_relu_inp,h_fc4, h_fc4_relu_inp, h_fc5, h_fc5_relu_inp, h_fc6, h_fc6_relu_inp, h_fc7, h_fc7_relu_inp, h_fc8, h_fc8_relu_inp, h_fc9, h_fc9_relu_inp, h_fc10, h_fc10_relu_inp, y_fc = create_model()
feed_dict = {x:mnist.test.images, y_: np.eye(10)[mnist.test.labels], keep_prob: 1.0, W_fc1: weightMatrix[0], b_fc1: biasMatrix[0], W_fc2: weightMatrix[1], b_fc2: biasMatrix[1], W_fc3: weightMatrix[2], b_fc3: biasMatrix[2], W_fc4: weightMatrix[3], b_fc4: biasMatrix[3], W_fc5: weightMatrix[4], b_fc5: biasMatrix[4], W_fc6: weightMatrix[5], b_fc6: biasMatrix[5],W_fc7: weightMatrix[6], b_fc7: biasMatrix[6],W_fc8: weightMatrix[7], b_fc8: biasMatrix[7],W_fc9: weightMatrix[8], b_fc9: biasMatrix[8], W_fc10: weightMatrix[9], b_fc10: biasMatrix[9], W_fc11: weightMatrix[10], b_fc11: biasMatrix[10]}
  
test_accuracy = accuracy.eval(feed_dict)
print("Test accuracy %g"%(test_accuracy))   



--2019-12-09 20:58:02--  https://raw.githubusercontent.com/safednn-nasa/prophecy_DNN/master/MNIST_ITR_REL/mnist_10_layer.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 146765 (143K) [text/plain]
Saving to: ‘./mnist_10_layer.txt’

./mnist_10_layer.tx 100%[===================>] 143.33K  --.-KB/s    in 0.04s   

2019-12-09 20:58:02 (3.65 MB/s) - ‘./mnist_10_layer.txt’ saved [146765/146765]

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Test accuracy 0.933


## Get tensors from pre-trained model

In [0]:
#cross_entropy, accuracy, x, keep_prob, y_fc, y_, W_fc1, W_fc2, W_fc3, W_fc4, W_fc5, W_fc6, W_fc7, W_fc8, W_fc9, W_fc10, W_fc11,b_fc1, b_fc2, b_fc3, b_fc4, b_fc5, b_fc6, b_fc7, b_fc8, b_fc9, b_fc10, b_fc11, h_fc1, h_fc2, h_fc2_relu_inp, h_fc3, h_fc3_relu_inp,h_fc4, h_fc4_relu_inp, h_fc5, h_fc5_relu_inp, h_fc6, h_fc7, h_fc8, h_fc9, h_fc10, y_fc = create_model()

cross_entropy, accuracy, x, keep_prob, y_fc, y_, W_fc1, W_fc2, W_fc3, W_fc4, W_fc5, W_fc6, W_fc7, W_fc8, W_fc9, W_fc10, W_fc11,b_fc1, b_fc2, b_fc3, b_fc4, b_fc5, b_fc6, b_fc7, b_fc8, b_fc9, b_fc10, b_fc11, h_fc1, h_fc1_relu_inp, h_fc2, h_fc2_relu_inp, h_fc3, h_fc3_relu_inp,h_fc4, h_fc4_relu_inp, h_fc5, h_fc5_relu_inp, h_fc6, h_fc6_relu_inp, h_fc7, h_fc7_relu_inp, h_fc8, h_fc8_relu_inp, h_fc9, h_fc9_relu_inp, h_fc10, h_fc10_relu_inp, y_fc = create_model()

t_fc1 = h_fc1 #sess.graph.get_tensor_by_name('h_fc1:0')
t_fc2 = h_fc2 #sess.graph.get_tensor_by_name('h_fc2:0')
t_fc3 = h_fc3 #sess.graph.get_tensor_by_name('h_fc3:0')
t_fc4 = h_fc4 #sess.graph.get_tensor_by_name('h_fc4:0')
t_fc5 = h_fc5 #sess.graph.get_tensor_by_name('h_fc5:0')
t_fc6 = h_fc6 #sess.graph.get_tensor_by_name('h_fc6:0')
t_fc7 = h_fc7 #sess.graph.get_tensor_by_name('h_fc7:0')
t_fc8 = h_fc8 #sess.graph.get_tensor_by_name('h_fc8:0')
t_fc9 = h_fc9 #sess.graph.get_tensor_by_name('h_fc9:0')
t_fc10 = h_fc10 #sess.graph.get_tensor_by_name('h_fc10:0')

if (LAYER == 1):
  curr_lay = t_fc1
  prev_lay = t_fc1
  curr_hlay = h_fc1
  prev_hlay = h_fc1
  inp_lay = h_fc1_relu_inp
  
if (LAYER == 2):
  curr_lay = t_fc2
  prev_lay = t_fc1
  curr_hlay = h_fc2
  prev_hlay = h_fc1
  inp_lay = h_fc2_relu_inp
  
if (LAYER == 3):
  curr_lay = t_fc3
  prev_lay = t_fc2
  curr_hlay = h_fc3
  prev_hlay = h_fc2
  inp_lay = h_fc3_relu_inp
  
if (LAYER == 4):
  curr_lay = t_fc4
  prev_lay = t_fc3
  curr_hlay = h_fc4
  prev_hlay = h_fc3
  inp_lay = h_fc4_relu_inp

if (LAYER == 5):
  curr_lay = t_fc5
  prev_lay = t_fc4
  curr_hlay = h_fc5
  prev_hlay = h_fc4
  inp_lay = h_fc5_relu_inp
  
if (LAYER == 6):
  curr_lay = t_fc6
  prev_lay = t_fc5
  curr_hlay = h_fc6
  prev_hlay = h_fc5
  inp_lay = h_fc6_relu_inp
  
if (LAYER == 7):
  curr_lay = t_fc7
  prev_lay = t_fc6
  curr_hlay = h_fc7
  prev_hlay = h_fc6
  inp_lay = h_fc7_relu_inp
  
if (LAYER == 8):
  curr_lay = t_fc8
  prev_lay = t_fc7
  curr_hlay = h_fc8
  prev_hlay = h_fc7
  inp_lay = h_fc8_relu_inp
  
if (LAYER == 9):
  curr_lay = t_fc9
  prev_lay = t_fc8
  curr_hlay = h_fc9
  prev_hlay = h_fc8
  inp_lay = h_fc9_relu_inp
  
if (LAYER == 10):
  curr_lay = t_fc10
  prev_lay = t_fc9
  curr_hlay = h_fc10
  prev_hlay = h_fc9
  inp_lay = h_fc10_relu_inp
  
t_label = tf.placeholder(tf.int32)
t_neuron_id = tf.placeholder(tf.int32)
t_grad = tf.gradients(y_fc[:, t_label], x)
t_grad_neuron = tf.gradients(y_fc[:, t_label], curr_hlay)[0]
#t_grad_conductance = tf.gradients(curr_lay[:,t_neuron_id], x, grad_ys=t_grad_neuron[:, t_neuron_id])
t_grad_conductance = tf.gradients(curr_lay[:,t_neuron_id], x)# grad_ys=t_grad_neuron[:, t_neuron_id]) ## WE DO NOT NEED HOW THE NEURON IMPACTS THE O/P


In [0]:
import copy

def get_prediction(inps, tensor=y_fc, batch_size=100):
  def get_prediction_batch(batch):
    #feed = {x: np.array(batch), keep_prob:1.0}
    feed = {x: np.array(batch), keep_prob:1.0, W_fc1: weightMatrix[0], b_fc1: biasMatrix[0], W_fc2: weightMatrix[1], b_fc2: biasMatrix[1], W_fc3: weightMatrix[2], b_fc3: biasMatrix[2], W_fc4: weightMatrix[3], b_fc4: biasMatrix[3], W_fc5: weightMatrix[4], b_fc5: biasMatrix[4], W_fc6: weightMatrix[5], b_fc6: biasMatrix[5],W_fc7: weightMatrix[6], b_fc7: biasMatrix[6],W_fc8: weightMatrix[7], b_fc8: biasMatrix[7],W_fc9: weightMatrix[8], b_fc9: biasMatrix[8], W_fc10: weightMatrix[9], b_fc10: biasMatrix[9], W_fc11: weightMatrix[10], b_fc11: biasMatrix[10]}
    return sess.run(tensor, feed_dict=feed)
  n = len(inps)
  if n%batch_size == 0:
    batches = [inps[i*batch_size:(i+1)*batch_size] for i in range(int(n/batch_size))]
  else:
    batches = [inps[i*batch_size:(i+1)*batch_size] for i in range(int(n/batch_size) +1)]    
  batch_predictions = [get_prediction_batch(b) for b in tqdm(batches)]
  return np.concatenate(tuple(batch_predictions), axis=0)

def attribute(inp, label, baseline=None, steps=50, use_top_label=False):
  def top_label(inp):
    return np.argmax(get_prediction([inp])[0])
  if baseline is None:
    baseline = 0*inp
  scaled_inputs = [baseline + (float(i)/steps)*(inp-baseline) for i in range(0, steps)]
  #feed = {keep_prob:1.0}
  feed = {keep_prob:1.0, W_fc1: weightMatrix[0], b_fc1: biasMatrix[0], W_fc2: weightMatrix[1], b_fc2: biasMatrix[1], W_fc3: weightMatrix[2], b_fc3: biasMatrix[2], W_fc4: weightMatrix[3], b_fc4: biasMatrix[3], W_fc5: weightMatrix[4], b_fc5: biasMatrix[4], W_fc6: weightMatrix[5], b_fc6: biasMatrix[5],W_fc7: weightMatrix[6], b_fc7: biasMatrix[6],W_fc8: weightMatrix[7], b_fc8: biasMatrix[7],W_fc9: weightMatrix[8], b_fc9: biasMatrix[8], W_fc10: weightMatrix[9], b_fc10: biasMatrix[9], W_fc11: weightMatrix[10], b_fc11: biasMatrix[10]}
    
  if use_top_label:
    feed[x] = [inp]
    logits = sess.run(y_fc, feed_dict=feed)[0]
    label = np.argmax(logits)
  feed[x] = scaled_inputs
  feed[t_label] = label
  grads, scores = sess.run([t_grad, y_fc], feed_dict=feed)  # shapes: <steps+1>, <steps+1, inp.shape>
  integrated_gradients = (inp-baseline)*np.average(grads[0], axis=0)  # shape: <inp.shape>
#  print "FINAL SCORE", scores[-1][label]
#  print "BASELINE SCORE", scores[0][label]
#  print "SUM", np.sum(integrated_gradients), "DIFF", scores[-1][label] - scores[0][label]
  return integrated_gradients

def conductance(inp, label, neuron_id=None, baseline=None, steps=50):
  # neuron_id is the id of the neuron in layer t_fc1 through which conductance
  # must be computed. If None, vanilla IG is computed.
  if baseline is None:
    baseline = 0*inp
  scaled_inputs = [baseline + (float(i)/steps)*(inp-baseline) for i in range(0, steps)]
  feed = {keep_prob:1.0, W_fc1: weightMatrix[0], b_fc1: biasMatrix[0], W_fc2: weightMatrix[1], b_fc2: biasMatrix[1], W_fc3: weightMatrix[2], b_fc3: biasMatrix[2], W_fc4: weightMatrix[3], b_fc4: biasMatrix[3], W_fc5: weightMatrix[4], b_fc5: biasMatrix[4], W_fc6: weightMatrix[5], b_fc6: biasMatrix[5],W_fc7: weightMatrix[6], b_fc7: biasMatrix[6],W_fc8: weightMatrix[7], b_fc8: biasMatrix[7],W_fc9: weightMatrix[8], b_fc9: biasMatrix[8], W_fc10: weightMatrix[9], b_fc10: biasMatrix[9], W_fc11: weightMatrix[10], b_fc11: biasMatrix[10]}
  feed[x] = scaled_inputs
  feed[t_label] = label
  if neuron_id != None:
    feed[t_neuron_id] = neuron_id
    grads, scores = sess.run([t_grad_conductance, y_fc], feed_dict=feed)  # shapes: <steps+1>, <steps+1, inp.shape>
    integrated_gradients = (inp-baseline)*np.average(grads[0], axis=0)  # shape: <inp.shape>
    return integrated_gradients
  grads, scores = sess.run([t_grad, y_fc], feed_dict=feed)  # shapes: <steps+1>, <steps+1, inp.shape>    
  integrated_gradients = (inp-baseline)*np.average(grads[0], axis=0)  # shape: <inp.shape>
  #print "FINAL SCORE", scores[-1][label]
  #print "BASELINE SCORE", scores[0][label]
  #print "SUM", np.sum(integrated_gradients), "DIFF", scores[-1][label] - scores[0][label]
  return integrated_gradients

## Library for Visualizing Images and Attributions

In [0]:
import PIL.Image
from IPython.display import clear_output, Image, display, HTML
import numpy as np
from io import StringIO
from io import BytesIO

In [0]:
FONT_PATH='/usr/share/fonts/truetype/dejavu/DejaVuSansCondensed.ttf'
IMAGE_SIZE = 28

def mnist_to_rgb(mnist_img):
  """
  Transformsn an MNIST image (shape: <784>) to a grayscale
  RGB image (shape: <28,28,3>)
  """
  pixel_array = mnist_img.reshape(IMAGE_SIZE, IMAGE_SIZE)  # shape: 28,28
  rgb_image = np.transpose([pixel_array,pixel_array,pixel_array], axes=[1,2,0])
  return rgb_image

def pil_img(a):
  '''Returns a PIL image created from the provided RGB array.
  '''
  a = np.uint8(a)
  return PIL.Image.fromarray(a)

def mnist_to_pil_img(inp):
  rgb_inp = 255*mnist_to_rgb(inp)
  vis_inp = pil_img(rgb_inp)
  return vis_inp  

def pil_fig(fig):
  # Returns a PIL image obtained from the provided PLT figure.
  buf = io.BytesIO()
  fig.savefig(buf, format='png')
  plt.close(fig)
  buf.seek(0)
  img = PIL.Image.open(buf)
  return img

def show_img(img, fmt='jpeg'):
  '''Displays the provided PIL image
  '''
  #f = StringIO()
  f = BytesIO()
  img.save(f, fmt)
  display(Image(data=f.getvalue()))
 
def show_mnist_img(mnist_img):
  show_img(pil_img(255*mnist_to_rgb(mnist_img)))
  
def gray_scale(img):
  '''Converts the provided RGB image to gray scale.
  '''
  img = np.average(img, axis=2)
  return np.transpose([img, img, img], axes=[1,2,0])

def normalize(attrs, ptile=99):
  '''Normalize the provided attributions so that they fall between
     -1.0 and 1.0.
  '''
  h = np.percentile(attrs, ptile)
  l = np.percentile(attrs, 100-ptile)
  return np.clip(attrs/max(abs(h), abs(l)), -1.0, 1.0)    

def pil_text(strs, shape, start_h=10, start_w=10, font_size=18, color=(0, 0, 0)):
  # Returns a PIL image with the provided text.
  img = pil_img(255*np.ones(shape))
  draw = PIL.ImageDraw.Draw(img)
  font = PIL.ImageFont.truetype(FONT_PATH, font_size)
  h = start_h
  for s in strs: 
    draw.text((start_w,h), s, fill=color, font=font)
    h = h + 30
  return img

def combine(imgs, horizontal=True):
  # Combines the provided PIL Images horizontally or veritically
  if horizontal:
    w = np.sum([img.size[0]+10 for img in imgs])
    h = np.max([img.size[1] for img in imgs])
  else:
    w = np.max([img.size[0] for img in imgs])
    h = np.sum([img.size[1]+10 for img in imgs])
  final_img = PIL.Image.new('RGB', (w, h), color='white')
  pos = 0
  for img in imgs:
    if horizontal:
      final_img.paste(im=img, box=(pos,0))
      pos = pos+img.size[0]+10
    else:
      final_img.paste(im=img, box=(0,pos))
      pos = pos+img.size[1]+10
  return final_img

def visualize_attrs(img, attrs, ptile=99):
  '''Visaualizes the provided attributions by first aggregating them
    along the color channel to obtain per-pixel attributions and then
    scaling the intensities of the pixels in the original image in
    proportion to absolute value of these attributions.

    The provided image and attributions must of shape (224, 224, 3).
  '''
  if np.sum(attrs) == 0.0:
    # print "Attributions are all ZERO"
    return pil_img(0*img)
  attrs = gray_scale(attrs)
  attrs = abs(attrs)
  attrs = np.clip(attrs/np.percentile(attrs, ptile), 0,1)
  vis = img*attrs
  return pil_img(vis)
  
  
R=np.array([255,0,0])
G=np.array([0,255,0])
B=np.array([0,0,255])
def visualize_attrs2(img, attrs, pos_ch=G, neg_ch=R, ptile=99):
  '''Visaualizes the provided attributions by first aggregating them
     along the color channel and then overlaying the positive attributions
     along pos_ch, and negative attributions along neg_ch.

     The provided image and attributions must of shape (224, 224, 3).
  '''
 
  if np.sum(attrs) == 0.0:
    # print "Attributions are all ZERO"
    return pil_img(0*img)
 
  attrs = gray_scale(attrs)
  attrs = normalize(attrs, ptile)   
  
  pos_attrs = attrs * (attrs >= 0.0)
  #pos_attrs = pos_attrs1 * (abs(pos_attrs1) >= threshold)
  neg_attrs = -1.0 * attrs * (attrs < 0.0)
  #neg_attrs = -1.0 * neg_attrs1 * (abs(neg_attrs1) >= threshold)
  attrs_mask = pos_attrs*pos_ch + neg_attrs*neg_ch
  vis = 0.3*gray_scale(img) + 0.7*attrs_mask
  return pil_img(vis)

## Extracting Invariant Candidates

In [0]:
def fingerprint_suffix(inps):
  # Below t_fc1 is the final fully connected layer of size 1024.
  return (get_prediction(inps, tensor=t_fc6)>0.0).astype('int')

def fingerprint_signature(inps,ten = t_fc1):
  # Below t_fc1 is the final fully connected layer of size 1024.
  return (get_prediction(inps, tensor=ten)>0.0).astype('int')

def fingerprint_prefix(inps):
  return (get_prediction(inps, tensor=tf.reshape(t_fc1, [-1, 14*14*32]))>0.0).astype('int')

In [0]:
# train_suffixes, train_predictions are in the same order
# as mnist.train.images. Henceforth when we use the index i we will
# be referring to mnist.train.images[i].
train_suffixes = fingerprint_signature(mnist.train.images, curr_lay)
print("Suffixes computed for all training data")
train_predictions = np.argmax(get_prediction(mnist.train.images), axis=1)
print("Predictions computed for all training data")

 12%|█▏        | 68/550 [00:00<00:00, 674.55it/s]

Suffixes computed for all training data


100%|██████████| 550/550 [00:00<00:00, 697.79it/s]

Predictions computed for all training data


In [0]:
def describe_input(i, training=True):
  print("Input", i)
  print("Groundtruth", mnist.train.labels[i])
  print("Prediction", train_predictions[i])
  print("Fine-grained prediction", 10*mnist.train.labels[i] + train_predictions[i])
  show_mnist_img(mnist.train.images[i])
  
def describe_input_INP(i):
  print("Input", i)
  print("Groundtruth", mnist_inp_labels[i])
  print("Prediction", inp_predictions[i])
  show_mnist_img(mnist_inp_images[i])

### Build the Decision Tree

In [0]:
# Basic decision tree
basic_estimator = tree.DecisionTreeClassifier()
basic_estimator.fit(train_suffixes, train_predictions)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [0]:
# Fine-grained predictions decision tree
#fine_grained_predictions = 10*mnist.train.labels + train_predictions
#fine_grained_estimator = tree.DecisionTreeClassifier()
#fine_grained_estimator.fit(train_suffixes, fine_grained_predictions)

In [0]:
# Decision tree per label
def get_relative_predictions(label):
  #print "Create relative predictions for label:%d" % label
  res = np.zeros(train_predictions.shape)
  for i in range(len(train_predictions)):
    pred = train_predictions[i]
    gt = mnist.train.labels[i]
    if gt == label and pred == gt:
      res[i] = 0
    elif gt == label and pred != gt:
      res[i] = 1
    else:
      res[i] = 2
  #print "Num correct: %d" % np.sum(res == 0)
  #print "Num misclassified: %d" % np.sum(res == 1)
  #print "Num others: %d" % np.sum(res == 2)
  return res

def get_relative_estimator(label):
  predictions = get_relative_predictions(label)
  #print "Creating decision tree for label:%d" % label
  estimator = tree.DecisionTreeClassifier()
  estimator.fit(train_suffixes, predictions)
  return estimator

In [0]:
# SLOW; run only if you want to build relative estimators.
#relative_estimators = [None for _ in range(10)]
#for i in range(10):
#  relative_estimators[i] = get_relative_estimator(i)
  

In [0]:
!wget https://raw.githubusercontent.com/safednn-nasa/prophecy_DNN/master/marabou1.elf -O ./marabou1.elf
!wget https://raw.githubusercontent.com/safednn-nasa/prophecy_DNN/master/MNIST_ITR_REL/mnist_10_layer.nnet -O ./mnist_10_layer.nnet
!pwd
!ls -lt ./marabou1.elf
!chmod 777 ./marabou1.elf
!ls -lt ./marabou1.elf

--2019-12-09 20:58:09--  https://raw.githubusercontent.com/safednn-nasa/prophecy_DNN/master/marabou1.elf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4231816 (4.0M) [application/octet-stream]
Saving to: ‘./marabou1.elf’

./marabou1.elf      100%[===================>]   4.04M  --.-KB/s    in 0.1s    

2019-12-09 20:58:09 (39.5 MB/s) - ‘./marabou1.elf’ saved [4231816/4231816]

--2019-12-09 20:58:11--  https://raw.githubusercontent.com/safednn-nasa/prophecy_DNN/master/MNIST_ITR_REL/mnist_10_layer.nnet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK

## INVOKE MARABOU FOR VERIFYING POTENTIAL RULES

In [0]:
def invoke_marabou_chk(layer,neurons,signature,label):
  #layer = 1
  #neurons = [4, 8, 7, 1, 0, 2, 5, 3, 9, 6] 
  #signature = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  #label = 6

  for lab_indx in range(0,10):
    if (lab_indx == label):
      continue
    strInp = ""
    for i in range(0,784):
      strInp = strInp + "x"+ str(i) + " >= 0.0" + "\n"
      #strInp = strInp + "x"+ str(i) + " <= 1.0" + "\n"
    #print(strInp)

    strInternal = ""
    for i in range(0,len(neurons)):
      strInternal = strInternal + "ws_"+ str(layer) + "_" + str(neurons[i])
      if (signature[i] == 0):
         strInternal = strInternal + " <= 0.0" + "\n"
      else:
         strInternal = strInternal + " >= 0.0"  + "\n"

    strOP = "-y"+ str(lab_indx) + " +y" + str(label) + " <= 0.00" + "\n"

    #Write to a property file
    file1 = open('property.txt',"w")
    #file1.writelines(strInp) 
    file1.writelines(strInternal) 
    file1.writelines(strOP) 
    file1.close() 

    file1 = open('property.txt',"r")  
    print("PROPERTY FILE IS ")
    print(file1.read())
    file1.close()

    !./marabou1.elf ./mnist_10_layer.nnet ./property.txt --summary-file=summary1.txt
    print("SUMMARY:")
    f = open('summary1.txt', 'r')
    file_contents = f.read()
    print (file_contents)
    f.close()
    if (file_contents.find('UNSAT') == -1):
        break
  #f.close()


### Examine clusters/invariants

In [0]:
def get_decision_path(estimator, inp):
  # Extract the decision path taken by an input as an ordered list of indices
  # of the neurons that were evaluated.
  # See: http://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html
  n_nodes = estimator.tree_.node_count
  feature = estimator.tree_.feature

  # First let's retrieve the decision path of each sample. The decision_path
  # method allows to retrieve the node indicator functions. A non zero element of
  # indicator matrix at the position (i, j) indicates that the sample i goes
  # through the node j.
  X_test = [inp]
  node_indicator = estimator.decision_path(X_test)
  # Similarly, we can also have the leaves ids reached by each sample.
  leaf_id = estimator.apply(X_test)
  # Now, it's possible to get the tests that were used to predict a sample or
  # a group of samples. First, let's make it for the sample.
  node_index = node_indicator.indices[node_indicator.indptr[0]:
                                      node_indicator.indptr[1]]
  neuron_ids = []
  for node_id in node_index:
    if leaf_id[0] == node_id:
        continue
    neuron_ids.append(feature[node_id])
  return neuron_ids

def get_suffix_cluster(neuron_ids, neuron_sig,suffixes=train_suffixes):
  # Get the cluster of inputs that such that all inputs in the cluster
  # have provided on/off signature for the provided neurons.
  #
  # The returned cluster is an array of indices (into mnist.train.images).
  return np.where((suffixes[:, neuron_ids] == neuron_sig).all(axis=1))[0]

def is_consistent_cluster(cluster, predictions):
  # Check if all inputs within the cluster have the same prediction.
  # 'cluster' is an array of input ids.
  pred = predictions[cluster[0]]
  for i in cluster:
    if predictions[i] != pred:
      return False
  return True

def is_misclassified(i):
  return train_predictions[i] != mnist.train.labels[i]

def visualize_conductances(img, label, neuron_ids, signature,only_on=False):
  # Visualize the conductances for the provided image.
  # Args:
  # - img: the provided mnist image
  # - label: prediction label w.r.t. conductance must be computed
  # - neuron_ids: list of neurons indices from the suffix tensor for which
  #    conductances must be computed.
  # - only_on: If True then conductance is computed only for those neurons
  #    that are on for the given image. 
  vis = [mnist_to_pil_img(img)]
  #suffix = fingerprint_signature([img],curr_lay)
  sumigc = 0.0
  for i, id in enumerate(neuron_ids):
    if only_on and signature[i] != 1:
      continue  
    igc = conductance(img, label, neuron_id=id)
    for indx in range(0,len(igc)):
      #print(indx,igc[indx]) ## a -ve gradient indicates the pixel value decreases the value of the neuron, making it go towards zero and negative
      if (signature[i] != 1): ## in a property if we want the o/p of a neuron to be zero, pixels which increase the neurons output should be given negative weightage and vice versa
         igc[indx] = -(igc[indx])
    sumigc = sumigc + igc 
  
  avgigc = sumigc / len(neuron_ids) ## gradient of each pixel w.r.t entire property - HIGHER VALUE INDICATES THE PIXEL HAS HIGHER CHANCE OF MAINTAINING THE PROPERTY
  maxval = abs(max(avgigc, key=abs))
  minval = abs(min(avgigc, key=abs))
  threshold = (maxval - minval)/2.0
  print("MAX ATR:", maxval, "MIN ATR:", minval, "THRESH:", threshold)
  avgigc = 1.0 * avgigc * (abs(avgigc) >= threshold) ## pixels with less significance in either SAT or DIS-SAT the property get blacked out
  
  
  vis.append(visualize_attrs2(255*mnist_to_rgb(img), mnist_to_rgb(avgigc)))
  return combine(vis)

def get_invariant_inp(estimator, ref_id, suffixes):
  # Returns an invariant found w.r.t. the provided reference input
  # Args
  #  - inp: reference input, shape <784,>
  # Returns:
  #  - cluster: Indices of training inputs that satisfy the invariant
  #  - neuron_id: A list of neurons such that all inputs that agree with
  #    the reference input on the on/off status of these neurons have the
  #    same prediction as the reference input.
  ref_img = mnist_inp_images[ref_id]
  ref_suffix = suffixes[ref_id]
  print('PREFIX',ref_suffix)
  neuron_ids = get_decision_path(estimator, ref_suffix)
  print('NEURON IDS',neuron_ids)
  neuron_sig = ref_suffix[neuron_ids]
  print('NEURON SIGNATURE',neuron_sig)
  cluster = get_suffix_cluster(neuron_ids, neuron_sig,suffixes)
  imgs = []
  cnt = 0
  for indx1 in range(0,len(cluster)):
    img = mnist.train.images(cluster[indx1])
    fnd = 1
    for i in range(0,len(img)):
      if (ref_img[i] != img[i]):
        fnd = 0
        break
    if (fnd == 1):
        ref_id = cnt
    cnt = cnt + 1
    imgs.append(img)
    
  imgs_suffixes = fingerprint_signature(imgs,t_fc2)
  ref_suffix = imgs_suffixes[ref_id]
  print('PREFIX',ref_suffix)
  neuron_ids = get_decision_path(estimator, ref_suffix)
  print('NEURON IDS',neuron_ids)
  neuron_sig = ref_suffix[neuron_ids]
  print('NEURON SIGNATURE',neuron_sig)
  cluster = get_suffix_cluster(neuron_ids, neuron_sig,imgs_suffixes)
    
  return cluster, neuron_ids, neuron_sig

def get_invariant(estimator, ref_id):
  # Returns an invariant found w.r.t. the provided reference input
  # Args
  #  - ref_id: Index (into mnist.train.images) of the reference input
  # Returns:
  #  - cluster: Indices of training inputs that satisfy the invariant
  #  - neuron_id: A list of neurons such that all inputs that agree with
  #    the reference input on the on/off status of these neurons have the
  #    same prediction as the reference input.
  ref_img = mnist.train.images[ref_id]
  ref_suffix = train_suffixes[ref_id]
  neuron_ids = get_decision_path(estimator, ref_suffix)
  neuron_sig = ref_suffix[neuron_ids]
  cluster = get_suffix_cluster(neuron_ids, neuron_sig)
  return cluster, neuron_ids, neuron_sig


def get_all_invariants(estimator):
  # Returns a dictionary mapping each decision tree prediction class
  # to a list of invariants. Each invariant is specified as a triple:
  # - neuron ids
  # - neuron signature (for the neuron ids)
  # - number of training samples that hit it
  # The neuron ids and neuron signature can be supplied to get_suffix_cluster
  # to obtain the cluster of training instances that hit the invariant.
  def is_leaf(node):
    return estimator.tree_.children_left[node] == estimator.tree_.children_right[node]

  def left_child(node):
    return estimator.tree_.children_left[node]

  def right_child(node):
    return estimator.tree_.children_right[node]
  
  def get_all_paths_rec(node):
    # Returns a list of triples corresponding to paths
    # in the decision tree. Each triple consists of
    # - neurons encountered along the path
    # - signature along the path
    # - prediction class at the leaf
    # - number of training samples that hit the path
    # The prediction class and number of training samples
    # are set to -1 when the leaf is "impure".
    feature = estimator.tree_.feature
    if is_leaf(node):
      values = estimator.tree_.value[node][0]
      if len(np.where(values != 0)[0]) == 1:
        cl = estimator.classes_[np.where(values != 0)[0][0]]
        nsamples = estimator.tree_.n_node_samples[node]
      else:
        # impure node
        cl = -1
        nsamples = -1
      return [[[], [], cl, nsamples]]
    # If it is not a leaf both left and right childs must exist
    paths = [[[feature[node]] + p[0], [0] + p[1], p[2], p[3]] for p in get_all_paths_rec(left_child(node))]
    paths += [[[feature[node]] + p[0], [1] + p[1], p[2], p[3]] for p in get_all_paths_rec(right_child(node))]
    return paths
  paths =  get_all_paths_rec(0)
  print("Obtained all paths")
  invariants = {}
  for p in tqdm(paths):
    neuron_ids, neuron_sig, cl, nsamples = p
    if cl not in invariants:
      invariants[cl] = []
    # cluster = get_suffix_cluster(neuron_ids, neuron_sig)
    invariants[cl].append([neuron_ids, neuron_sig, nsamples])
  for cl in invariants.keys():
    invariants[cl] = sorted(invariants[cl], key=operator.itemgetter(2), reverse=True)
  return invariants


def describe_cluster(cluster, neuron_ids, show_samples=False):
  neuron_sig = train_suffixes[cluster[0]][neuron_ids]
  print("Num neurons in invariant", len(neuron_ids))
  print("Neuron id and signature")
  
  for i in range(0,len(neuron_ids)):
    print("id:", neuron_ids[i], "sig:", neuron_sig[i])
  
  print("Cluster size: ", len(cluster))
  print("Num misclassified", len([i for i in cluster if is_misclassified(i)]))
  if show_samples:
    for i in range(10):
      images = []
      for j in range(10):
        if 10*i + j >= len(cluster):
          break
        images.append(mnist_to_pil_img(mnist.train.images[cluster[10*i+j]]))
      if len(images) > 0:
        show_img(combine(images))
  

def describe_invariants_all_labels(all_invariants,prevlayer = t_fc1,layer = t_fc2,suffixes=train_suffixes,COMMON=False, DEC_PREFX= False):
  
  print("PRINTING PURE RULES WITH SUPPORT MORE THAN 50 FOR EVERY LABEL:");
  for cl, invs in all_invariants.items():
    if (cl == -1):
      continue
    
    for indx in range (0, len(invs)):
      inv = invs[indx]
      cls = get_suffix_cluster(inv[0],inv[1],suffixes)
      
      neurons = inv[0]
      signature = inv[1]

      if (len(cls) <= 50):
        continue
      print("Class:", cl, ", Rule:(neurons:",inv[0],",signature:",inv[1],"), Support:",inv[2],", Num misclassified", len([i for i in cls if is_misclassified(i)]));

      print("PIXELS IMPACTING PROPERTY (conductance) for 10 inputs satisfy the property")
      interval = int(len(cls)/10)
      print("INTERVAL:", interval)
      i = 0
      while (i < len(cls)):
        ref_id = cls[i]
        if (is_misclassified(ref_id)):
          print("MISCLASSIFIED")
        else:
          print("CORRECTLY CLASSIFIED")
        show_img(visualize_conductances(mnist.train.images[ref_id], train_predictions[ref_id], inv[0], inv[1],only_on=False))
        i = i + interval

      #invoke_marabou_chk(LAYER,neurons,signature,cl)

      if (COMMON == True):
          common_nodes(cls,suffixes)

      if (DEC_PREFX == True):
          decision_prefs(cls,suffixes)

  return
  
def common_nodes(cls,suffixes):
    cnt = 0
    common = np.zeros(10,dtype=int)
    prev = np.zeros(10,dtype=int)
    
    for indx in range(0, len(cls)):
        i = cls[indx]
        cnt = cnt + 1
        for j in range(0,len(suffixes[i])):
          if (common[j] == -1):
             continue
          if ((indx != 0) and (suffixes[i][j] != prev[j])):
             common[j] = -1
          else:
             common[j] = suffixes[i][j]
          prev[j] = suffixes[i][j]


    print('COMMON NODES IN CLUSTER for CLASS:',cl,cnt)
    com = []
    for k in range(0,len(common)):
        if (common[k] != -1):
           com.append((k,common[k]))
    print(com)

    return
    
def decision_prefs(cls,suffixes):
    images = mnist.train.images
    imgsCom = []
    imgs = []
    for indx in range(0, len(cls)):
        print('IMG:')
        print(list(zip(images[cls[indx]])))
        imgs.append(images[cls[indx]])
        imgsCom.append(images[cls[indx]])
            
    dec_prefixes= fingerprint_signature(imgs,layer)
    prefixes = []
    for indx in range(0,len(dec_prefixes)):
       dec_pref = dec_prefixes[indx]
    
       match = 0
       for indx1 in range(0, len(prefixes)):
          match = 1
          for i in range(0,len(prefixes[indx1])):
             if (dec_pref[i] != prefixes[indx1][i]):
                match = 0
                break
          if (match == 1):
             break
    
       if (match == 0):
          prefixes.append(dec_pref)
    
    print('DECISION PREFIXES IN CLUSTER for CLASS:',cl,cnt)
    for k in range(0,len(prefixes)):
      print(prefixes[k])

    return
    
  
  #print('LAYER INPS:')
  #min = np.zeros(10)
  #max = np.zeros(10)
  #for dim in range(0,10):
  #    min[dim] = 1000
  #    max[dim] = -1000
          
  #prevlayer_vals = get_prediction(imgsCom,prevlayer)      
  #print('MIN, MAX LAYER INPS:',len(prevlayer_vals))
  #for i in range(0,len(prevlayer_vals)):
  #    if (i == 0):
  #      print(zip(prevlayer_vals[i]))
  #    for dim in range(0,10):
  #        if ( prevlayer_vals[i][dim] < min[dim]):
  #            min[dim] = prevlayer_vals[i][dim]
  #        if ( prevlayer_vals[i][dim] > max[dim]):
  #            max[dim] = prevlayer_vals[i][dim]
    
  #print('MIN')
  #print(zip(min))
  #print('MAX')
  #print(zip(max))    
    
  #df = pd.DataFrame(df, columns=['Prediction Class', 'Num Instances', 'Num Invariants', 'Num Invariants with cluster size >= 10', 'Size of largest invariant cluster'])
  #df = pd.DataFrame(df,columns=['Pred Class','Total #Neurons','# Invariants'])
  #return df


def describe_all_invariants(all_invariants):
  df = []
  for cl, invs in all_invariants.iteritems(): 
    inv = invs[0]
    clus = get_suffix_cluster(inv[0],inv[1])
    #print(len(clus))
    misCl = 0
    for i in range(0,len(clus)):
      indx = clus[i]
      if (is_misclassified(indx) == True):
        misCl = misCl + 1
    print('class:',cl,',masSup:',inv[2],',#misCl:',misCl)
          

In [0]:
print("layer:", LAYER)
#",label:", LABEL)
invariants = get_all_invariants(basic_estimator)
describe_invariants_all_labels(invariants,prev_lay,curr_lay)


#ref_id = 3
#print("### Cluster ###")
#cluster, neuron_ids, neuron_sig = get_invariant(basic_estimator, ref_id)
#describe_cluster(cluster, neuron_ids, show_samples=True)

# Visualize  10 inputs in the cluster
#for i in cluster[:10]:
#show_img(visualize_conductances(mnist.train.images[ref_id], train_predictions[ref_id], neuron_ids, only_on=False))
    
#print "###  BASIC DECISION TREE ###"




## Print invariant stats
#describe_all_invariants(invariants,t_fc1,t_fc1)
#describe_all_invariantsFull(invariants,True,t_fc2)
#print "Total num invariants:", df['Num Invariants'].sum()
#print "Total num invariants with cluster size >= 10:", df['Num Invariants with cluster size >= 10'].sum()
#print df.to_string(index=False)

### OTHER INPUTS ####
#for indxi in range(0,len(mnist_inp_images)):
#    inp = mnist_inp_images[indxi]
#    lab = mnist_inp_labels[indxi] 
#    print(indxi, lab)
#    describe_input_INP(indxi)
##    print "###  BASIC DECISION TREE Cluster ###"
#    if (indxi == 0):
#       cluster, neuron_ids, neuron_sig = get_invariant_inp(basic_estimator,indxi )
#       print('LEN CLUSTER:',len(cluster),is_consistent_cluster(cluster,train_predictions))
 #  # describe_cluster(cluster, neuron_ids)

#    print "###  RELATIVE DECISION TREE Cluster ###"
#    cluster, neuron_ids, neuron_sig = get_invariant(relative_estimators[lab], indxi)
#    describe_cluster(cluster, neuron_ids)
   
    
#print "###  BASIC DECISION TREE ###"
#invariants = get_all_invariants(basic_estimator)
## Print invariant stats
#describe_all_invariants1(invariants)
#describe_all_invariantsFull(invariants,True,t_fc2)
#print "Total num invariants:", df['Num Invariants'].sum()
#print "Total num invariants with cluster size >= 10:", df['Num Invariants with cluster size >= 10'].sum()
#print df.to_string(index=False)


#for cl in invariants.keys():
#  print(cl,invariants[cl])
  
#print "###  RELATIVE DECISION TREE for CLASS 0"
#invariants = get_all_invariants(relative_estimators[0])
#df = describe_all_invariants(invariants)
##print "Total num invariants:", df['Num Invariants'].sum()
##print "Total num invariants with cluster size >= 10:", df['Num Invariants with cluster size >= 10'].sum()
#print df.to_string(index=False)
##for cl in invariants.keys():
##   print(cl,invariants[cl])

#print "###  RELATIVE DECISION TREE for CLASS 1"
#invariants = get_all_invariants(relative_estimators[1])
#df = describe_all_invariants(invariants)
##print "Total num invariants:", df['Num Invariants'].sum()
##print "Total num invariants with cluster size >= 10:", df['Num Invariants with cluster size >= 10'].sum()
#print df.to_string(index=False)
##for cl in invariants.keys():
##   print(cl,invariants[cl])
    
#print "###  RELATIVE DECISION TREE for CLASS 2"
#invariants = get_all_invariants(relative_estimators[2])
#df = describe_all_invariants(invariants)
##print "Total num invariants:", df['Num Invariants'].sum()
##print "Total num invariants with cluster size >= 10:", df['Num Invariants with cluster size >= 10'].sum()
#print df.to_string(index=False)
##for cl in invariants.keys():
##   print(cl,invariants[cl])
    
#print "###  RELATIVE DECISION TREE for CLASS 3"
#invariants = get_all_invariants(relative_estimators[3])
#df = describe_all_invariants(invariants)
##print "Total num invariants:", df['Num Invariants'].sum()
##print "Total num invariants with cluster size >= 10:", df['Num Invariants with cluster size >= 10'].sum()
#print df.to_string(index=False)
##for cl in invariants.keys():
##   print(cl,invariants[cl])
    
#print "###  RELATIVE DECISION TREE for CLASS 4"
#invariants = get_all_invariants(relative_estimators[4])
#df = describe_all_invariants(invariants)
##print "Total num invariants:", df['Num Invariants'].sum()
##print "Total num invariants with cluster size >= 10:", df['Num Invariants with cluster size >= 10'].sum()
#print df.to_string(index=False)
##for cl in invariants.keys():
##   print(cl,invariants[cl])
    
#print "###  RELATIVE DECISION TREE for CLASS 5"
#invariants = get_all_invariants(relative_estimators[5])
#df = describe_all_invariants(invariants)
##print "Total num invariants:", df['Num Invariants'].sum()
##print "Total num invariants with cluster size >= 10:", df['Num Invariants with cluster size >= 10'].sum()
#print df.to_string(index=False)
##for cl in invariants.keys():
##   print(cl,invariants[cl])

#print "###  RELATIVE DECISION TREE for CLASS 6"
#invariants = get_all_invariants(relative_estimators[6])
#df = describe_all_invariants(invariants)
##print "Total num invariants:", df['Num Invariants'].sum()
##print "Total num invariants with cluster size >= 10:", df['Num Invariants with cluster size >= 10'].sum()
#print df.to_string(index=False)
##for cl in invariants.keys():
##   print(cl,invariants[cl])
    
#print "###  RELATIVE DECISION TREE for CLASS 7"
#invariants = get_all_invariants(relative_estimators[7])
#df = describe_all_invariants(invariants)
##print "Total num invariants:", df['Num Invariants'].sum()
##print "Total num invariants with cluster size >= 10:", df['Num Invariants with cluster size >= 10'].sum()
#print df.to_string(index=False)
##for cl in invariants.keys():
##   print(cl,invariants[cl])
    
#print "###  RELATIVE DECISION TREE for CLASS 8"
#invariants = get_all_invariants(relative_estimators[8])
#df = describe_all_invariants(invariants)
##print "Total num invariants:", df['Num Invariants'].sum()
##print "Total num invariants with cluster size >= 10:", df['Num Invariants with cluster size >= 10'].sum()
#print df.to_string(index=False)
#for cl in invariants.keys():
#   print(cl,invariants[cl])
    
#print "###  RELATIVE DECISION TREE for CLASS 9"
#invariants = get_all_invariants(relative_estimators[9])
#df = describe_all_invariants(invariants)
##print "Total num invariants:", df['Num Invariants'].sum()
##print "Total num invariants with cluster size >= 10:", df['Num Invariants with cluster size >= 10'].sum()
#print df.to_string(index=False)
##for cl in invariants.keys():
##   print(cl,invariants[cl])







100%|██████████| 67/67 [00:00<00:00, 44876.78it/s]

layer: 4
Obtained all paths
PRINTING PURE RULES WITH SUPPORT MORE THAN 50 FOR EVERY LABEL:
Class: 4 , Rule:(neurons: [0, 5, 3, 6, 7, 8] ,signature: [1, 1, 0, 0, 0, 0] ), Support: 919 , Num misclassified 2
PIXELS IMPACTING PROPERTY (conductance) for 10 inputs satisfy the property
INTERVAL: 91
CORRECTLY CLASSIFIED
MAX ATR: 0.07542468 MIN ATR: 0.0 THRESH: 0.03771233931183815


CORRECTLY CLASSIFIED
MAX ATR: 0.07673863 MIN ATR: 0.0 THRESH: 0.0383693166077137


CORRECTLY CLASSIFIED
MAX ATR: 0.084175736 MIN ATR: 0.0 THRESH: 0.042087867856025696


CORRECTLY CLASSIFIED
MAX ATR: 0.06925056 MIN ATR: 0.0 THRESH: 0.034625280648469925


...

CORRECTLY CLASSIFIED
MAX ATR: 0.12749913 MIN ATR: 0.0 THRESH: 0.06374956667423248


CORRECTLY CLASSIFIED
MAX ATR: 0.07838277 MIN ATR: 0.0 THRESH: 0.03919138386845589


CORRECTLY CLASSIFIED
MAX ATR: 0.11149993 MIN ATR: 0.0 THRESH: 0.05574996396899223


CORRECTLY CLASSIFIED
MAX ATR: 0.17382888 MIN ATR: 0.0 THRESH: 0.08691444247961044


CORRECTLY CLASSIFIED
MAX ATR: 0.26275018 MIN ATR: 0.0 THRESH: 0.13137508928775787


In [0]:
## Get all fine_grained_estimator invariants
#fge_all_invariants = get_all_invariants(fine_grained_estimator)
## Print invariant stats
#df = describe_all_invariants(fge_all_invariants)
#print "Total num invariants:", df['Num Invariants'].sum()
#print "Total num invariants with cluster size >= 10:", df['Num Invariants with cluster size >= 10'].sum()
#print df.to_string(index=False)



### Analyzing clusters of misclassified inputs

In [0]:
## Examine the cluster for a misclasification (Groundtruth: 4, Prediction: 49)
#invs = fge_all_invariants[49]
#neuron_ids, neuron_sig, _ = invs[0]
#cluster = get_suffix_cluster(neuron_ids, neuron_sig)
#describe_cluster(cluster, neuron_ids)

## Visualize  10 inputs in the cluster
#for i in cluster[:10]:
#  describe_input(i)
 # # show_img(visualize_conductances(mnist.train.images[i], train_predictions[i], neuron_ids, only_on=False))

### Test Accuracy Improvements

In [0]:
## We use the fine_grained_estimator to check if an input belongs
## to a pure cluster (i.e., prediction id of the form 10*label + label).
## If so, we declare the network's prediction as a "condident prediction".
## We measure the accuracy of confident_predictions.
#fine_grained_estimator_test_predictions = fine_grained_estimator.predict(test_suffixes)
#fine_grained_estimator_leaf_nodes = fine_grained_estimator.apply(test_suffixes)

In [0]:
def test_accuracy_for_label(label):
  def get_confidence():
    is_confident = (fine_grained_estimator_test_predictions == 10*label + label)
    sufficient_samples = fine_grained_estimator.tree_.n_node_samples[fine_grained_estimator_leaf_nodes] >= 10
    is_confident *= sufficient_samples
    return is_confident
  # Following are boolean array. For e.g., with_label[i] is True if
  # image i has the given label
  with_label = (mnist.test.labels == label)
  is_correct = (test_predictions == mnist.test.labels)
  with_label_and_correct = with_label*is_correct
  is_confident = get_confidence()
  with_label_and_correct_and_confident = with_label_and_correct*is_confident
  with_label_and_confident = with_label*is_confident

  total = np.sum(with_label)
  num_conf = np.sum(with_label_and_confident) 
  num_correct = np.sum(with_label_and_correct)
  num_correct_conf = np.sum(with_label_and_correct_and_confident)
  return total, num_conf, num_correct, num_correct_conf

In [0]:
#df = []
#grand_total = 0
#grand_correct = 0
#grand_conf = 0
#grand_correct_conf = 0
#for i in range(10):
#  total, num_conf, num_correct, num_correct_conf = test_accuracy_for_label(i)
#  grand_total += total
#  grand_conf += num_conf
#  grand_correct += num_correct
#  grand_correct_conf += num_correct_conf
#  acc = 1.0*num_correct/total
#  conf_acc = 1.0*num_correct_conf/num_conf
#  df.append([i, total, num_conf, acc, conf_acc])
#df = pd.DataFrame(df, columns=['Label', 'Instances', 'ConfidentInstances',  'Acc', 'ConfidentAcc',])
#display(df)
#print "Total Instances", grand_total
#print "Num Confident Instances", grand_conf
#print "Orig Accuracy", 1.0*grand_correct/grand_total
#print "Confident Accuracy", 1.0*grand_correct_conf/grand_conf

### Visualizing the Decision Tree

In [0]:
#!apt-get install graphviz
#!pip install graphviz
#import graphviz

In [0]:
#dot_data = tree.export_graphviz(basic_estimator, out_file=None) 
#graph = graphviz.Source(dot_data)  
#graph 